## One-hot Comparison 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import dataset into a pandas data frame

df = pd.read_csv('../41467_2022_32829_MOESM5_ESM.csv')
df.head()

,ID,DNA1 [counts],DNA2 [counts],DNA3 [counts],RNA1 [counts],RNA2 [counts],RNA3 [counts],TX1 [au],TX2 [au],TX3 [au],...,high quality,Observed log(TX/Txref),Predicted log(TX/Txref),dG10,dG35,dGDisc,dGITR,dGEXT10,dGSPAC,dGUP
0,0,8263,7261,5173,16341,10320,13506,2.258071,1.523795,1.545541,...,Yes,-3.386326,-3.844827,-1.781524,-1.477218,-0.106428,-0.021112,0.191352,-0.0924,0.400862
1,1,5600,4886,3264,10986,7250,10800,2.240001,1.590845,1.958709,...,Yes,-3.503140,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
2,2,7898,6790,4752,19572,32204,30585,2.829533,5.084911,3.810029,...,Yes,-4.207206,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
3,3,10651,9875,6466,15734,16246,18908,1.686729,1.763814,1.731036,...,Yes,-3.392439,-3.877808,-1.781524,-1.477218,-0.139409,-0.021112,0.191352,-0.0924,0.400862
4,4,12188,10793,6965,28609,21796,26803,2.680198,2.165100,2.278025,...,Yes,-3.698903,-3.672384,-1.781524,-1.477218,0.066015,-0.021112,0.191352,-0.0924,0.400862


In [ ]:
# All input and output data

X = df[['UP', 'h35', 'spacs', 'h10', 'disc', 'ITR']]
y = df['Observed log(TX/Txref)']

X.head()

,UP,h35,spacs,h10,disc,ITR
0,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CCCCGCGG,CTCTACCTTAGTTTGTACGTT
1,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGCGGCGG,CTCTACCTTAGTTTGTACGTT
2,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGCGCCCG,CTCTACCTTAGTTTGTACGTT
3,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,GCGGCGGC,CTCTACCTTAGTTTGTACGTT
4,TTTTCTATCTACGTAC,TTGACA,CTATTTCCTATTTCTCT,TATAAT,CGGGGGGC,CTCTACCTTAGTTTGTACGTT


In [ ]:
# Function to one-hot encode DNA sequences

def encode(sequence):
    mapping = {'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1]}
    encoding = []
    for nucleotide in sequence:
         encoding += [mapping[nucleotide]]
    return encoding

In [ ]:
# Function to one-hot encode DNA sequences

def dummy_drop_encode(sequence):
    mapping = {'A': [1,0,0], 'C': [0,1,0], 'G': [0,0,1], 'T': [0,0,0]}
    encoding = []
    for nucleotide in sequence:
         encoding += [mapping[nucleotide]]
    return encoding

In [ ]:
# stores the various input approaches
X_dict = {}

# stores split training/testing
train_test = {}

# stores the results
results = {}

# stores the models
models = {}

# stores the model history
model_history = {}

In [ ]:
# Concatenate the one-hot encoded h35 + h10 sequence motifs
X_dict['no_dummy'] = np.array([encode(h35 + h10) for h35, h10 in zip(df['h35'], df['h10'])])

# The first entry for this approach, one-hot encoded from 'TTGACATATAAT'
X_dict['no_dummy'][0]

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1]])

In [ ]:
# split the data in training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_dict['no_dummy'], y, test_size=0.2, random_state=1, shuffle=True)
train_test['no_dummy'] = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}


In [ ]:
# Import necessary libraries
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
def build_model(current_model):
    # Define RNN model architecture
    models[current_model] = Sequential()
    models[current_model].add(LSTM(64, input_shape=X_dict[current_model].shape[1:])) # dynamically generated input shape based on X data
    models[current_model].add(Dense(1, activation='linear'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    models[current_model].compile(optimizer=optimizer, loss='mean_squared_error')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history = models[current_model].fit(train_test[current_model]['X_train'],
                                    train_test[current_model]['y_train'],
                                    epochs=150,
                                    batch_size=32,
                                    validation_data=(X_test, y_test),
                                    callbacks=[early_stopping])

    # Evaluate the model
    loss = models[current_model].evaluate(train_test[current_model]['X_test'], train_test[current_model]['y_test'])

    return models[current_model], loss, history

In [14]:
# Call the function to build the model, save the model, and store the results

m = 'no_dummy'

models[m], results[m], model_history[m] = build_model(m)
models[m].save(m + '.keras')

Epoch 1/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.2557 - val_loss: 0.3109
Epoch 2/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3323 - val_loss: 0.2493
Epoch 3/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2427 - val_loss: 0.2104
Epoch 4/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2230 - val_loss: 0.2417
Epoch 5/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2240 - val_loss: 0.2015
Epoch 6/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2163 - val_loss: 0.2033
Epoch 7/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2192 - val_loss: 0.2158
Epoch 8/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2184 - val_loss: 0.1988
Epoch 9/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2178 - val_loss: 0.2015
Epoch 10/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2166 - val_loss: 0.2023
Epoch 11/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2137 - val_loss: 0.2007
Epoch 12/150
337/337 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Concatenate the one-hot encoded h35 + h10 sequence motifs
X_dict['dummy_drop'] = np.array([dummy_drop_encode(h35 + h10) for h35, h10 in zip(df['h35'], df['h10'])])

# The first entry for this approach, one-hot encoded from 'TTGACATATAAT'
X_dict['dummy_drop'][0]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0]])

In [ ]:
# split the data in training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_dict['dummy_drop'], y, test_size=0.2, random_state=1, shuffle=True)
train_test['dummy'] = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

In [ ]:
# Call the function to build the model, save the model, and store the results

m = 'dummy_drop'

models[m], results[m], model_history[m] = build_model(m)
models[m].save(m + '.keras')

Epoch 1/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.5073 - val_loss: 0.3413
Epoch 2/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3216 - val_loss: 0.2168
Epoch 3/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2316 - val_loss: 0.2190
Epoch 4/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2282 - val_loss: 0.2201
Epoch 5/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2327 - val_loss: 0.2043
Epoch 6/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2345 - val_loss: 0.2252
Epoch 7/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2188 - val_loss: 0.2052
Epoch 8/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2110 - val_loss: 0.2061
Epoch 9/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2170 - val_loss: 0.2181
Epoch 10/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2196 - val_loss: 0.2064
Epoch 11/150
337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2196 - val_loss: 0.2016
Epoch 12/150
337/337 ━━━━━━━━━━━━━━━━━━━━